In [ ]:
#automatically reload modules when they have changed
%reload_ext autoreload
from pathlib import Path
import random
import os

from yaml_config_wrapper import Configuration
from RLcraft import MalmoMazeEnv

In [ ]:
yml_path = 'configs/mazes.yml'
# Load YML config file
c = Configuration(config_src=yml_path)
# Load configs from config class
c_general = c.get_config('general')[0]
c_tuner = c.get_config('tuner')[0]
# Load the values from the config
run_config = c_tuner['config']
env_config = run_config['env_config']
c_general = c_general['config']

In [ ]:
try:
    env.close()
except:
    pass
# Generate a seed for maze 
print("Generating new seed ...")
maze_seed = random.randint(1, 9999)
print("Loading environment ...")
# num = random.randint(0, 8)
# mission_file = run_config["env_config"]["mission_file"].replace("*", str(num))
# from glob import glob
# mission_files = glob(run_config["env_config"]["mission_file"])
# mission_file = random.choice(mission_files)
os.environ['MALMO_XSD_PATH'] = '/Users/gkos/Insync/delfinas7kostas@gmail.com/Google Drive/Projects/UTK/Minecraft-AI/MalmoPlatform/Schemas'

env = MalmoMazeEnv(
            width=600,
            height=600,
            mazeseed=maze_seed,
            xml=env_config["mission_file"],
            millisec_per_tick=env_config['millisec_per_tick'],
            max_loop=c_general['max_loop'],
            mission_timeout_ms=c_general['mission_timeout_ms'],
            step_reward=c_general['step_reward'],
            win_reward=c_general['win_reward'],
            lose_reward=c_general['lose_reward'],
            action_space=c_general['action_space'],
            client_port=env_config['client_port'],
            time_wait=c_general['time_wait'])
print("Resetting environment ...")
_ = env.reset()
print("Environment Loaded!")

In [ ]:
print("Possible actions and action space actions for current env:")
for action in range(6):
    print(f"Action: {action} -> {env.action_space[action]}")

In [ ]:
print("Sample an action")
action = env.action_space.sample()
print(f'{action} -> {env.action_space[action]}')

In [ ]:
# Take an action and get the observation, reward, done, and info
obs, reward, done, info = env.step(3)

In [ ]:
print("Reward: ", reward)
print("Done: ", done)
print("Info: ", info)
print(f"Observation: type={type(obs)}, shape={obs.shape}")

In [ ]:
print(info.observations[0])

In [ ]:
# Use the internal actions to move the agent
# move: walks forwards/backwards
#   0 -> Nothing, 1 -> Forward, -1 -> Backward
# strafe: walks left/right
#   0 -> Nothing, 1 -> Right, -1 -> Left
# turn: turns the camera left/right without moving
#   0 -> Nothing, 1 -> Right, -1 -> Left
env.agent_host.sendCommand("turn 1")

In [15]:
env.close()